In [31]:
import os
import joblib
import numpy as np
import matplotlib.pyplot as plt
from model_settings import ms
from pathlib import Path
ms.find_root(Path())
models_dir = os.path.join(ms.root,ms.trained_models)
models = [f for f in os.listdir(models_dir) if f.find('.')==-1]
for i,m in enumerate(models):
    print(f"{i}  {m}")
# i = int(input('select model: '))
i=-1
model_dir = os.path.join(models_dir,models[i])
pickle = os.path.join(model_dir,models[i]+'.pkl')

def test_performance(model,net):
    inpred = net.predict(model['train_X'])
    intar = model['train_data'][model['target_name']].values
    indiff = inpred-intar
    inRMSE = np.sqrt(np.mean(indiff**2))
    inMAE = np.mean(np.abs(indiff))
    print(f"in sample\n   RMSE:\n      {inRMSE}")
    print(f"   MAE:\n      {inMAE}")

    outpred = net.predict(model['test_X'])
    outtar = model['test_data'][model['target_name']].values
    outdiff = outpred-outtar
    outRMSE = np.sqrt(np.mean(outdiff**2))
    outMAE = np.mean(np.abs(outdiff))
    print(f"out of sample\n   RMSE:\n      {outRMSE}")
    print(f"   MAE:\n      {outMAE}")

0  2024-12-03 115353819604 inital cboe spx relative asian
1  2024-12-03 221708375538 inital cboe spx relative barrier
2  2024-12-06 024553040968 inital cboe spx relative barrier


In [32]:
model = joblib.load(pickle)

In [33]:
model['mlp_params']

{'solver': 'sgd',
 'activation': 'relu',
 'max_iter': 500,
 'random_state': 710,
 'early_stopping': True,
 'hidden_layer_sizes': (20,),
 'learning_rate': 'adaptive',
 'learning_rate_init': 0.1,
 'n_iter_no_change': 20,
 'tol': 0.0001,
 'warm_start': False}

In [34]:
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline

In [35]:
mlp = model['fitted']
mlp[1].out_activation_ = 'relu'

In [36]:
test_performance(model,mlp)

in sample
   RMSE:
      0.04982492956128657
   MAE:
      0.024180624363508776
out of sample
   RMSE:
      0.3635187406668067
   MAE:
      0.18823574538357962


In [28]:
import pandas as pd
param_df = pd.DataFrame()
for k,v in mlp[1].__dict__.items():
    try:
        param_df.loc[str(k),'specification'] = str(v)
    except Exception:
        pass

param_df = param_df[:31].copy()
param_df = param_df.drop(index=[i for i in param_df.index if i.startswith('_')]).reset_index()
param_df = param_df.rename(columns={'index':'parameter'})
latex_table = param_df.to_latex(index=False, escape=False)
print(latex_table)

\begin{tabular}{ll}
\toprule
parameter & specification \\
\midrule
activation & relu \\
solver & sgd \\
alpha & 0.0001 \\
batch_size & auto \\
learning_rate & adaptive \\
learning_rate_init & 0.1 \\
power_t & 0.5 \\
max_iter & 500 \\
loss & squared_error \\
hidden_layer_sizes & (10,) \\
shuffle & True \\
random_state & 1312 \\
tol & 0.0001 \\
verbose & False \\
warm_start & False \\
momentum & 0.9 \\
nesterovs_momentum & True \\
early_stopping & True \\
validation_fraction & 0.1 \\
beta_1 & 0.9 \\
beta_2 & 0.999 \\
epsilon & 1e-08 \\
n_iter_no_change & 20 \\
max_fun & 15000 \\
n_features_in_ & 17 \\
n_outputs_ & 1 \\
n_iter_ & 299 \\
t_ & 71833554 \\
n_layers_ & 3 \\
out_activation_ & relu \\
\bottomrule
\end{tabular}

